In [1]:
from QCNN.model import Net
import torch
import torch.nn as nn


In [2]:
PATH = 'cifar_net.pth'

net = Net()
net.load_state_dict(torch.load(PATH, weights_only=True))

<All keys matched successfully>

In [3]:
net.eval()

Net(
  (net): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): Linear(in_features=400, out_features=120, bias=True)
    (6): ReLU()
    (7): Linear(in_features=120, out_features=84, bias=True)
    (8): ReLU()
    (9): Linear(in_features=84, out_features=10, bias=True)
  )
)

In [12]:
from torch.ao.quantization import convert

In [13]:
net.qconfig = torch.ao.quantization.get_default_qconfig('qnnpack')
model_prepared = torch.ao.quantization.prepare(net)

In [14]:
import torchvision
import torchvision.transforms as transforms
transform = transforms.Compose([
        transforms.ToTensor(),  # Convert PIL Image to Tensor
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize RGB channels
    ])

# Set hyperparameters
batch_size = 4  # Number of samples per batch

# Initialize CIFAR-10 training dataset
trainset = torchvision.datasets.CIFAR10(
    root='./data',  # Dataset storage directory
    train=True,  # Use training split
    download=True,  # Download if missing
    transform=transform
)

# Create training data loader with multiprocessing
trainloader = torch.utils.data.DataLoader(
    trainset,
    batch_size=batch_size,
    shuffle=True,  # Shuffle samples each epoch
    num_workers=2  # Subprocesses for data loading
)

In [17]:
with torch.no_grad():
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        model_prepared(inputs)

In [18]:
model_quantized = convert(model_prepared)

In [19]:
torch.save(model_quantized.state_dict(), 'quantized.pth')

In [26]:
model_quantized.modules()

<generator object Module.modules at 0x127f4a190>

In [32]:
q_model = dict(net.named_modules())

In [29]:
q_named_layers = dict(model_quantized.named_modules())

In [35]:
criterion = nn.CrossEntropyLoss()

In [41]:
model_quantized.eval()

Net(
  (net): Sequential(
    (0): QuantizedConv2d(3, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.049589142203330994, zero_point=139)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=0.07314004004001617, zero_point=125)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Flatten(start_dim=1, end_dim=-1)
    (5): QuantizedLinear(in_features=400, out_features=120, scale=0.13583976030349731, zero_point=129, qscheme=torch.per_tensor_affine)
    (6): ReLU()
    (7): QuantizedLinear(in_features=120, out_features=84, scale=0.10098320245742798, zero_point=151, qscheme=torch.per_tensor_affine)
    (8): ReLU()
    (9): QuantizedLinear(in_features=84, out_features=10, scale=0.1033138781785965, zero_point=107, qscheme=torch.per_tensor_affine)
  )
)

In [45]:
avg_loss = 0.0
with torch.no_grad():
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        out = model_quantized(inputs)
        loss = criterion(out, labels)
        avg_loss += loss.item()

NotImplementedError: Could not run 'quantized::conv2d.new' with arguments from the 'CPU' backend. This could be because the operator doesn't exist for this backend, or was omitted during the selective/custom build process (if using custom build). If you are a Facebook employee using PyTorch on mobile, please visit https://fburl.com/ptmfixes for possible resolutions. 'quantized::conv2d.new' is only available for these backends: [MPS, Meta, QuantizedCPU, BackendSelect, Python, FuncTorchDynamicLayerBackMode, Functionalize, Named, Conjugate, Negative, ZeroTensor, ADInplaceOrView, AutogradOther, AutogradCPU, AutogradCUDA, AutogradXLA, AutogradMPS, AutogradXPU, AutogradHPU, AutogradLazy, AutogradMTIA, AutogradMeta, Tracer, AutocastCPU, AutocastMTIA, AutocastXPU, AutocastMPS, AutocastCUDA, FuncTorchBatched, BatchedNestedTensor, FuncTorchVmapMode, Batched, VmapMode, FuncTorchGradWrapper, PythonTLSSnapshot, FuncTorchDynamicLayerFrontMode, PreDispatch, PythonDispatcher].

MPS: registered at /Use